In [10]:
import torch

class ClearCache:
    def __enter__(self):
        torch.cuda.empty_cache()

    def __exit__(self, exc_type, exc_val, exc_tb):
        torch.cuda.empty_cache()



In [11]:
import os
import os
import time
import warnings
import json
# from faster_whisper import WhisperModel
# import whisper
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
with ClearCache():

    os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
    cache_dir = r'..\HF_cache'

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id = "openai/whisper-large-v3"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True, _attn_implementation='flash_attention_2',
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=30,
        batch_size=16,
        return_timestamps=True,
        torch_dtype=torch_dtype,
        generate_kwargs={"language": "english"},
        model_kwargs={"attn_implementation": "flash_attention_2"},
        device=device
    )
    
    # pipe(r"C:\Users\jvish\OneDrive\Documents\VISH_Stuff\VW_Proj_backend\MultiModal\media\video.mp3")
    

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
torch.cuda.empty_cache()

In [12]:
import gc
del pipe
del model
del processor
del pipeline
gc.collect

<function gc.collect(generation=2)>

In [13]:
torch.cuda.empty_cache()